In [1]:
import os
import keras_tuner
import tensorflow as tf

from tensorflow import keras

In [2]:
base_data_dir = os.path.join('..', 'data', 'baseline_data')
train_data_dir = os.path.join(base_data_dir, 'training')
valid_data_dir = os.path.join(base_data_dir, 'validation')

train_ds = keras.utils.image_dataset_from_directory(
    directory=train_data_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(600, 450))
valid_ds = keras.utils.image_dataset_from_directory(
    directory=valid_data_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(600, 450))

Found 8015 files belonging to 7 classes.
Found 2000 files belonging to 7 classes.


In [3]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [4]:
num_classes = 7


def build_model_with_prelu(hp) -> tf.keras.Sequential:
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255, input_shape=(600, 450, 3)),
        tf.keras.layers.Resizing(width=300, height=225, crop_to_aspect_ratio=True),
        tf.keras.layers.Conv2D(
            hp.Choice('filters_l1', [64, 96, 128]),
            hp.Choice('kernel_size_l1', [7, 8]),),
        tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(value=0.25)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(
            hp.Choice('filters_l2', [32, 64, 96]),
            hp.Choice('kernel_size_l2', [5, 6]),
            activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(
            hp.Choice('filters_l3', [16, 32]),
            hp.Choice('kernel_size_l3', [3, 4]),
            activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hp.Choice('units_l4', [128, 256])),
        tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(value=0.25)),
        tf.keras.layers.Dense(num_classes),
        tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(value=0.25))
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    return model

In [5]:
tuner_with_prelu = keras_tuner.BayesianOptimization(
    build_model_with_prelu,
    objective='val_accuracy')

In [6]:
tuner_with_prelu.search(train_ds, epochs=5, validation_data=valid_ds)

Trial 1 Complete [00h 11m 50s]
val_accuracy: 0.11100000143051147

Best val_accuracy So Far: 0.11100000143051147
Total elapsed time: 00h 11m 50s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
128               |128               |filters_l1
8                 |7                 |kernel_size_l1
32                |96                |filters_l2
5                 |5                 |kernel_size_l2
32                |32                |filters_l3
3                 |3                 |kernel_size_l3
256               |128               |units_l4

Epoch 1/5
251/251 [==============================] - 133s 519ms/step - loss: 5.1195 - accuracy: 0.6672 - val_loss: 5.0853 - val_accuracy: 0.6705
Epoch 2/5
251/251 [==============================] - 128s 510ms/step - loss: 5.1019 - accuracy: 0.6692 - val_loss: 5.0853 - val_accuracy: 0.6705
Epoch 3/5
251/251 [==============================] - 127s 506ms/step - loss: 5.1019 - accuracy: 0.6692 - val_loss: 5.0853 - val_accu

KeyboardInterrupt: 

In [10]:
best_params = tuner_with_prelu.get_best_hyperparameters()[0]


In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(600, 450, 3)),
    tf.keras.layers.Conv2D(128, 8, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 6, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 4, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(value=0.25)),
    tf.keras.layers.Dense(num_classes),
    tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(value=0.25))
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [20]:
model.fit(train_ds, validation_data=valid_ds, epochs=5)

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_5/max_pooling2d_15/MaxPool/MaxPoolGrad' defined at (most recent call last):
    File "C:\ProgramData\anaconda3\envs\pracainz\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\anaconda3\envs\pracainz\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\anaconda3\envs\pracainz\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\anaconda3\envs\pracainz\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\ProgramData\anaconda3\envs\pracainz\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\ProgramData\anaconda3\envs\pracainz\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.r